In [ ]:
#= Costanti (sola lettura)
-- Numbers and boolean --
number_of_steps = 1000; # Number of steps to execute in simulation, Intero positivo [1,+∞)
dim = 3; # Dimensions of simulation, dimensioni dello spazio 2D o 3D, Intero positivo (tipicamente [2,3])
box_size = 10.0; # Size of one side of box, lunghezza di un lato del campo di simulazione Float positivo [1.0, +∞)
finite_box = true; # Whether box if finite or just where particles are initially placed, vero se il campo è finito<APPROFONDIRE
periodic = true; # Whether simulation is periodic < true se la simulazione è periodica (pac-man)
part_num = 100; # Number of particles in simulation, numero di particelle da simulare 
dt = 0.01; # Time step, controlla la precisione della simulazione
num_part_types = 2; # Number of types of particles, numero di tipologie di particelle Intero positivo [1, +∞]
save_interval = 10; # Ogni quanti passi viene salvata la posizione, (per la visualizzazione), Intero positivo [1, number_of_steps)

-- Arrays --
interaction_params = gen_interaction(num_part_types); # Interations parameters for types of particles, parametri di interazione, array?
mass_parts = rand(Float64, num_part_types); # Masses of types of particles, massa per ogni tipo di particella, array
part_types = rand(1:num_part_types, part_num) # Randomly assign type of each particle
=#

In [ ]:
#= Strutture dati (lettura, scrittura)

# Matrici bi-dimensionali per tracciare lo stato del sistema ad ogni step
pos = box_size*rand(Float64, part_num, dim) # Initialized to be randomly placed within a box
vel = zeros(Float64, part_num, dim) # Initialized to zero
acc = zeros(Float64, part_num, dim) # Initialized to zero
force = find_force(....) # 

# Matrici tridimensionali per salvare lo stato del sistema ad ogni intervallo
saved_positions = zeros(num_pos, part_num, dim) # Save positions with specified frequency, unico utile per la visualizzazione
saved_velocities = zeros(num_pos, part_num, dim) # Save velocities with specified frequency
saved_accelerations = zeros(num_pos, part_num, dim) # Save accelerations with specified frequency
saved_forces = zeros(num_pos, part_num, dim) # Save forces with specified frequency
=#

In [24]:
# Generate strengths of interactions between particle types
# Very simple but user could replace with anything they wanted
# Genera le intensità delle interazioni fra tipologie di particelle
# Modifiche: Cambio tipo da Float64 a Float32, inserito break perché matrice simmetrica
function gen_interaction(num_part_types)
    interaction_params = Array{Float32}(undef, num_part_types, num_part_types)
    for i=1:num_part_types
        for j = 1:num_part_types
            if (i==j) # Self-interaction is randomly repulsive
                interaction_params[i,j] = -1*rand(Float32)
            elseif (i<j) # Others randomly attractive
                val = rand(Float32)
                interaction_params[i,j] = val
                interaction_params[j,i] = val
            else
                break # Inserito break, arresta il ciclo, matrice simmetrica
            end
        end        
    end    
    return interaction_params
end

gen_interaction (generic function with 1 method)

In [33]:
# Test #
gen_interaction(3);
@time gen_interaction(3)

  0.000001 seconds (4 allocations: 272 bytes)


3×3 Array{Float32,2}:
 -0.565291  0.955043     0.899324   
  0.955043  4.26989e-29  0.0        
  0.899324  0.0          4.26991e-29

In [26]:
# Calcola le distanza tra due punti monodimensionali, nel caso sia un sistema periodico calcola anche la distanza passando dalla parte opposta.
function monodistances(x, y, box_size, periodic)
    # Se il sistema è periodico si può considerare
    # solo l'effetto della forza generata dalla particella a distanza minore
    # oppure la somma delle forze esercitate dalla particella su entrambe le distanze
    dist = abs(x - y)
    pdist = periodic ? box_size - dist : 0.        
    return dist, pdist
end

#= Calcola la forza agente su una particella con massa = mass1 
esercitata da una particella con massa = mass2 a distanza = dist 
e forza di interazione = int_strenght =#
function monoforce(mass1, mass2, int_strength, dist)
    # return int_strength / dist^2 # Formula del codice originale    
    return (int_strength * mass1 * mass2) / dist^2 # Formula per forza di gravità (se int_strength = costante di gravitazione universale G)  
end

#= PRE: Ogni punto si trova all'interno del box (se box finito) garantito dalla funzione step_update! che gestisce 
le posizioni uscenti dal box ad ogni iterazione. =#
# Find force on each particle
# 1/r^2 interactions: Is very simple but user can replace with anything they want
function find_force!(force, part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size)
    # Aggiunta: Azzera forze, molto più veloce azzerare lo stesso array che crearne uno nuovo ad ogni step, parallelizzabile
    zeroing!(force)
    # Per ogni particella i
    for i = 1:part_num # For every particle
        # Calcola la massa di i
        mass1 = mass_parts[part_types[i]] # La massa non veniva usata nel codice originale!
        # Per ogni altra particella k diversa da i
        for k = 1:part_num # Contribution from every other particle
            if i == k continue end # Modifica: Solo per ridurre l'indentazione, non è necessario incapsulare tutto il codice che segue.
            #if (i != k) # No self-interaction
            # Modifica: int_strength portato fuori dal loop sottostante
            int_strength = interaction_params[part_types[i], part_types[k]] # Strength of interaction between particles
            # Concettualmente nel ciclo sottostante (per ciascuna dimensione) andrebbe chiamata la sola funzione
            # per il calcolo della forza su ciascuna dimensione dopo aver ottenuto le grandezze necessarie:
            # distanza fra particelle, massa e forza di interazione.
            mass2 = mass_parts[part_types[k]] # Massa della particella agente            
            # Per ciascuna dimensione
            for j = 1:dim # For each dimension
                #= Il codice originale considera le forze agenti su ciascuna particella in modo strano:
                    Se il sistema è periodico prevale la distanza massima tra quella normale (contenuta nel box)
                    e quella periodica (seguendo il tagitto "pac-man style"), oltre al fatto che penso si debba invece
                    considerare quella minima o entrambe simultaneamente, credo che sia meglio calcolare le forze 
                    delle sole distanze contenute nel box a prescindere dal fatto che il sistema sia periodico o meno, 
                dato che ogni particella che supera il confine viene "teletrasportata" dal lato opposto. =#                    
                
                #= Find distance between particles    
                if (pos[i, j] > pos[k, j]) # Questo controllo può essere evitato senza calcolare il valore assoluto della distanza
                    int_strength = -1*int_strength # Reverses direction of force if positions flopped
                end
                    
                dist = 0.0
                if (periodic) # If periodic, check whether a periodic distance is shorter
                    dist = abs(pos[i,j] - pos[k,j])
                    if (pos[i,j] < pos[k,j])
                        new_dist = abs(box_size + pos[i,j] - pos[k,j])
                        if new_dist > dist
                            dist = new_dist
                            int_strength = -1*int_strength # Reverses direction of force
                        end
                    else
                        new_dist = abs(box_size + pos[k,j] - pos[i,j])
                        if new_dist > dist
                            dist = new_dist
                            int_strength = -1*int_strength # Reverses direction of force
                        end
                    end
                else # Otherwise, just regular distance
                    dist = abs(pos[i,j] - pos[k,j])
                end
                =#
                # Siccome la verlet prevede la scomposizione del calcolo per singola componente
                # La distanza va calcolata su ogni singola componente (unidimensionale)
                # Considera che nel caso si voglia usare la distanza geodedica ad esempio,
                # la curvatura comporta solo che per ciascuna componente il valore sarà >= alla distanza geodedica
                # pertanto verrà considerata solo la semplice distanza a - b per singola componente
                dist, pdist = monodistances(pos[i, j], pos[i, k], box_size, periodic)
                
                if periodic
                    # Nella formula finale si dovrebbe usare anche la massa di una particella
                    #force[i,j] += int_strength / dist^2 # 1/r^2 interaction # Modifica: accumula le forze per una sola componente
                    # Modifica, funzione ausiliaria per il calcolo della forza agente su una componente:
                    force[i, j] += monoforce(mass1, mass2, int_strength, min(dist, pdist)) # Forza agente sulla particella i per la componente j
                    
                    #experimental, aggiunge anche forza dalla direzione opposta
                    double_component_enabled = false
                    if double_component_enabled
                        force[i, j] += monoforce(mass1, mass2, int_strength, max(dist, pdist))
                    end
                
                else
                    force[i, j] += monoforce(mass1, mass2, int_strength, dist)
                end                    
            end
            #end del vecchio if per escludere l'auto-interazione
        end
    end
    return nothing # Modifica C++ style
end

find_force! (generic function with 1 method)

In [27]:
# Update position, velocity, and acceleration using Velocity Verlet Algorithm
# Can deal with infinite and finite systems
# For finite system, can be periodic or can reflect off walls
function step_update!(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic)

    # Loop is completely parallelizale
    for i = 1:part_num # For every particle
        mass = mass_parts[part_types[i]]
        for j = 1:dim # For each dimension
            pos[i,j] = pos[i,j] + vel[i,j]*dt + 0.5*acc[i,j]*dt^2 # x(t+Δt) = x(t) + v(t)Δt + 1/2*a(t)(Δt)^2
            vel[i,j] = vel[i,j] + 0.5*(acc[i,j] + force[i,j]/mass)*dt # v(t+Δt) = v(t) + 1/2*(a(t)+a(t+Δt))Δt
            acc[i,j] = force[i,j]/mass # a = F/m
            
            if (finite_box) # If finite box, check are still inside and correct if not
                if (periodic) # For periodic, just change position to be in box
                    if (pos[i,j] < 0) # If no longer in box
                        pos[i,j] = box_size + (pos[i,j] % box_size)
                    elseif (pos[i,j] > box_size) # If no longer in box
                        pos[i,j] = pos[i,j] % box_size                        
                    end
                else # If not periodic, more complicated - reflects off walls
                    if (pos[i,j] < 0) # If no longer in box
                        pos[i,j] = -1*(pos[i,j])
                    elseif (pos[i,j] > box_size) # If no longer in box
                        pos[i,j] = box_size - pos[i,j]
                    end
                    vel[i,j] = -1*(vel[i,j])
                    acc[i,j] = -1*(acc[i,j])
                end
            end
        end
    end    
    # return pos, vel, acc # No need to return b/c is passed by reference
    return nothing # Modifica, C++ style
end

step_update! (generic function with 1 method)

In [28]:
#= The main program function, take in some parameters, returns a 3D matrix of positions for each time interval.

-> Input:
1. number_of_steps = Numero di iterazioni della simulazione, intero positivo [1, +∞)
2. dim = Numero di dimensioni dello spazio (2D o 3D), intero positivo, tipicamente [2,3]
3. box_size = Lunghezza di un lato del box di simulazione (quadrato o cubo), float positivo [1.0, +∞)
4. finite_box = Vero se il campo è finito <APPROFONDIRE, booleano [true, false]
5. periodic = Vero se la simulazione è periodica (pac-man), booleano [true, false]
6. part_num = Numero di particelle da simulare, intero positivo [1, +∞) 
7. dt = Grandezza del passo temporale, float positivo preferibilmente piccolo (0, +∞)
8. num_part_types = Numero di tipologie di particelle, intero positivo [1, +∞)
9. save_interval = Ogni quanti steps viene salvata la posizione, intero positivo [1, number_of_steps]
# Nuove variabili (?)
10. load_from_disk = Vero se si vuole caricare da disco i dati di una simulazione precedente (sovrascrive alcuni parametri)
11. save_on_disk = Vero se si vuole salvare su disco lo stato finale della simulazione

<- Output:
1. saved_positions = Sequenza temporale delle posizioni di ciascuna particella, matrice 3D, num_pos x part_num x dim
(num_pos = Numero di posizioni da salvare, funzione di number_of_steps e save_interval )
============================#
function dynamics_simulation(number_of_steps, dim, box_size, finite_box, periodic, part_num, dt, num_part_types, save_interval)
    # Inizializza strutture dati    
    # Costanti dipendenti dai parametri
    interaction_params = gen_interaction(num_part_types) # parametri di interazione, matrice quadrata num_part_types^2
    mass_parts = rand(Float32, num_part_types); # Massa per ogni tipo di particella, array di Float32
    part_types = rand(1:num_part_types, part_num) # Tipologia di particella per ogni particella, array di Int
      
    # Variabili aggiornate ad ogni iterazione
    pos = box_size.*rand(Float32, part_num, dim) # Initialized to be randomly placed within a box (Usare funzione?)
    vel = Array{Float32}(undef, part_num, dim) #zeros(Float64, part_num, dim) # Initialized to zero
    acc = Array{Float32}(undef, part_num, dim) #zeros(Float64, part_num, dim) # Initialized to zero
    force = Array{Float32}(undef, part_num, dim) #zeros(Float64, part_num, dim) # Initialized to zero
    
    # Variabili per il salvataggio
    num_pos = floor(Int, number_of_steps/save_interval) + 1
    saved_positions = Array{Float32}(undef, num_pos, part_num, dim) #zeros(num_pos, part_num, dim)
    saved_velocities = Array{Float32}(undef, num_pos, part_num, dim) #zeros(num_pos, part_num, dim) # Save velocities with specified frequency
    saved_accelerations = Array{Float32}(undef, num_pos, part_num, dim) #zeros(num_pos, part_num, dim) # Save accelerations with specified frequency
    saved_forces = Array{Float32}(undef, num_pos, part_num, dim) #zeros(num_pos, part_num, dim) # Save forces with specified frequency
    
    # Calcolo delle forze agenti su ciascuna particella
    # Modifica: calcolo prima le forze e poi aggiorno con verlet direttamente nel loop
    # Loop per ciascuno step
    for t in 1:number_of_steps
        # Calcola forze agenti su ciascuna particella
        find_force!(force, part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size)
        # aggiornamento step
        step_update!(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic)        
        # Salvataggi
        # Salva posizioni (TODO: funzione per il salvataggio save!(m2, m3) m2 matrice 2D, m3 matrice 3D
        # Se salvi su disco salva anche vel, acc e force
    end
    # Restituisci le posizioni per intervallo di tempo
    return saved_positions
end

dynamics_simulation (generic function with 1 method)

In [31]:
dynamics_simulation(1000, 3, 10.0, true, true, 20, 0.01, 3, 2);
@time dynamics_simulation(100, 3, 10.0, true, true, 20, 0.01, 3, 2);

  0.000471 seconds (16 allocations: 50.984 KiB)


In [32]:
using BenchmarkTools
@benchmark dynamics_simulation(100, 3, 10.0, true, true, 20, 0.01, 3, 2)

BenchmarkTools.Trial: 
  memory estimate:  50.83 KiB
  allocs estimate:  12
  --------------
  minimum time:     477.300 μs (0.00% GC)
  median time:      867.399 μs (0.00% GC)
  mean time:        876.389 μs (0.21% GC)
  maximum time:     6.359 ms (0.00% GC)
  --------------
  samples:          5692
  evals/sample:     1

In [7]:
# Per le particelle i e k sulla componente dimensionale j
function aux_v1(posij, poskj, int_strength, periodic, box_size)
    if (posij > poskj)
        int_strength = -1*int_strength # Reverses direction of force if positions flopped
    end
    dist = 0.0
    if (periodic) # If periodic, check whether a periodic distance is shorter
        dist = abs(posij - poskj)
        if (posij < poskj)
            new_dist = abs(box_size + posij - poskj)
            if new_dist > dist
                dist = new_dist
                int_strength = -1*int_strength # Reverses direction of force
            end
        else
            new_dist = abs(box_size + poskj - posij)
            if new_dist > dist
                dist = new_dist
                int_strength = -1*int_strength # Reverses direction of force
            end
        end
    else # Otherwise, just regular distance
    dist = abs(posij - poskj)
    end
    return dist, int_strength
end


function aux_v2(posij, poskj, int_strength, periodic, box_size)
    # Force Direction
    
    # Distance
    dist = abs(posij - poskj)
    if periodic && box_size - dist >
        # Calcola anche la distanza periodica
        new_dist = box_size - dist
    
    
end

aux_v1 (generic function with 3 methods)

In [8]:
aux_v1(4, 7, 0.6, true, 10)

(7, -0.6)